In [ ]:
import pandas as pd

df = pd.read_csv('data/merged_data.csv')


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.optimize as sco

# Load the data
df = pd.read_csv('data/merged_data.csv')

# Function to calculate the Sharpe ratio
def sharpe_ratio(weights, returns):
    # Create the portfolio returns
    portfolio_return = np.dot(returns, weights)
    # Calculate mean and standard deviation of portfolio returns
    mean_return = np.mean(portfolio_return)
    std_return = np.std(portfolio_return)
    return -mean_return / std_return  # Negative because we minimize in optimization

# List of valid columns (after filtering by p-value)
valid_columns = []

# Loop to filter out columns with high p-values and prepare for optimization
for column in df.columns:
    data = df[column].dropna()
    data = data[np.isfinite(data)]
    data = data[np.abs(data) < 1e10]

    if data.empty:
        print(f"Skipping column {column} due to invalid data.")
        continue

    X = np.array(data.index).reshape(-1, 1)
    Y = np.array(data).reshape(-1, 1)

    X = sm.add_constant(X)

    # Fit model on the entire dataset
    model = sm.OLS(Y, X).fit()

    # Filter columns with p-value below 0.1
    p_values = model.pvalues
    if p_values[1] >= 0.20:  # Assuming the constant has index 0 and the feature has index 1
        print(f"Skipping column {column} due to high p-value: {p_values[1]:.4f}")
        continue

    valid_columns.append(column)  # Store valid column

# Prepare returns for the valid columns
returns = df[valid_columns].pct_change().dropna()  # Calculate daily returns (percentage change)

# Initial guess for the weights (equally weighted)
init_weights = np.ones(returns.shape[1]) / returns.shape[1]

# Constraint: Weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Bounds for each weight: between -0.1 and 0.1
bounds = [(-0.1, 0.1) for _ in range(returns.shape[1])]

# Optimize weights to maximize the Sharpe ratio
result = sco.minimize(sharpe_ratio, init_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=constraints)

# Extract optimized weights
optimized_weights = result.x

# Display the optimized weights
print("Optimized weights for each valid column:")
for i, weight in enumerate(optimized_weights):
    print(f"Column {valid_columns[i]}: {weight:.4f}")

# Calculate the optimized portfolio return and standard deviation
optimized_portfolio_return = np.dot(returns, optimized_weights)
optimized_sharpe_ratio = -result.fun

# Display results
print(f"\nOptimized Sharpe Ratio: {optimized_sharpe_ratio:.4f}")

# Plot Actual vs Predicted and Cumulative Sum for the valid columns
for column in valid_columns:
    data = df[column].dropna()
    data = data[np.isfinite(data)]
    data = data[np.abs(data) < 1e10]

    X = np.array(data.index).reshape(-1, 1)
    Y = np.array(data).reshape(-1, 1)

    X = sm.add_constant(X)

    # Fit model on the entire dataset
    model = sm.OLS(Y, X).fit()

    Y_pred = model.predict(X)

    cum_sum_actual = np.cumsum(Y)
    cum_sum_pred = np.cumsum(Y_pred)

    # Plot Actual vs Predicted and Cumulative Sum
    # plt.figure(figsize=(12, 8))

    # Actual vs Predicted plot
    # plt.subplot(2, 1, 1)
    # plt.scatter(X[:, 1], Y, color='blue', label='Actual', alpha=0.6)
    # plt.scatter(X[:, 1], Y_pred, color='red', label='Predicted', alpha=0.6)
    # plt.title(f'Actual vs Predicted - {column}')
    # plt.xlabel('Index')
    # plt.ylabel('Value')
    # plt.legend()
    # plt.grid(True)

    # # Cumulative Sum plot
    # plt.subplot(2, 1, 2)
    # plt.plot(cum_sum_actual, color='blue', label='Actual Cumulative Sum')
    # plt.plot(cum_sum_pred, color='red', label='Predicted Cumulative Sum')
    # plt.title(f'Cumulative Sum - {column}')
    # plt.xlabel('Index')
    # plt.ylabel('Cumulative Sum')
    # plt.legend()
    # plt.grid(True)

    # plt.tight_layout()
    # plt.show()

# Final optimized Sharpe ratio for the portfolio
print(f"Optimized Sharpe Ratio for the portfolio: {optimized_sharpe_ratio:.4f}")


In [51]:
valid_columns

['strat_2',
 'strat_3',
 'strat_7',
 'strat_9',
 'strat_11',
 'strat_14',
 'strat_15',
 'strat_19',
 'strat_23',
 'strat_24',
 'strat_26']